In [1]:
 import sympy
from sympy import Matrix, S, Symbol, symbols, I, zeros, eye
from sympy import simplify, expand, expand_complex, latex
import numpy as np
from IPython.display import Latex

# Практическое занятие 12
# Компьютерный практикум по алгебре на Python
## Матричные разложения: Холецкого, LDL, LU, QR.

## Скелетное разложение
## Пример 1
Построим скелетное разложение для матрицы
$$
\left(
\begin{matrix}
2 & 3 & -1 & 4\\
-1 & 4 & 1 & 4\\
1 & 7 & 0 & 8
\end{matrix}
\right)
$$
$B$ матрица полного столбцового ранга, $C$-строчного

In [2]:
A1 = Matrix([[2, 3, -1, 4], [-1, 4, 1, 4], [1, 7, 0, 8]])
A1_rref = A1.rref()
display(*A1_rref)

Matrix([
[1, 0, -7/11,  4/11],
[0, 1,  1/11, 12/11],
[0, 0,     0,     0]])

(0, 1)

Каждый элемент кортежа, возвращаемого rref стал отдельным аргументом функции display и был отрисован.

rref возвращает кортеж, первый элемент которого - ступенчатый вид матрицы, второй - кортеж из номеров ведущих столбцов матрицы.

Для получения матрицы полного столбцового ранга выделим из матрицы $A$ ведущие столбцы (в соответствии с кортежем номеров столбцов, полученным благодаря rref).



In [3]:
cols = A1_rref[1]
k = len(cols)
B = A1[:, cols]
display(Latex(f'cols = {latex(cols)}, k = {k}, B = {latex(B)}'))

<IPython.core.display.Latex object>

Для выделения нужных столбцов из матрицы воспользуемся срезом:
```
B = A1[:, cols]
```
В квадратных скобках указываем номера нужных строк (все строки обозначаются :) и номера столбцов. Номера могут быть диапазонами (например, 2:6),кортежами (например, (3, 6, 8)), списками, а еще можно использовать range.

Выделим из $A1_{rref}$ ненулевые строки, получим матрицу $C$ полного строчного ранга.


In [4]:
C = A1_rref[0][:k, :]
display(Latex(f'C = {latex(C)}'))

<IPython.core.display.Latex object>

Проверим, что построено разложение исходной матрицы:

In [5]:
display(Latex(fr'A = {latex(A1)}\quad BC = {latex(B * C)}'))

<IPython.core.display.Latex object>

**Симметричная** (вещественная) матрица $A$: $A^T = A$.

**Эрмитова** (комплексная) матрица $A$: $A^H = A$, т.е. матрица совпадает с транспонированной матрицей из комплексно-сопряженных чисел, например, матрица
$\left(
\begin{matrix}
12&3+ I\\
3 - I&2
\end{matrix}
\right)$

**Положительно определенная матрица** $A$: все угловые миноры положительны, например
$$
A=\left(
\begin{matrix}
4&6&10\\
6&13&17\\
10&17&62
\end{matrix}
\right), \qquad
\Delta_1 = |4| = 4 > 0, \qquad
\Delta_2 = \left|\begin{matrix}
4&6\\
6&13
\end{matrix}\right| = 16 > 0, \qquad
\Delta_3 = \left|\begin{matrix}
4&6&10\\
6&13&17\\
10&17&62
\end{matrix}\right| = 576 > 0
$$
### Разложение Холецкого
$A = L\cdot L^T$ для симметричной вещественной матрицы $A$

$A = L\cdot L^H$ для положительно определенной эрмитовой матрицы $A$

$L$ - левая треугольная матрица.

В Sympy в классе матриц есть метод **cholesky**, возвращающий матрицу $L$. В случае вещественной симметричной матрицы $A$ передаем методу необязательный параметр hermitian=False, по умолчанию равный True.

Поскольку разложение Холецкого применяется только к симметричным или эрмитовым положительно определенным матрицам, то сначала следует проверить, является ли матрица положительно определенной, это делается с помощью атрибута **is_positive_definite** класса матриц.

### Пример 2.
Построим разложение Холецкого матриц
$$
A=\left(
\begin{matrix}
4&6&10\\
6&13&17\\
10&17&62
\end{matrix}
\right)
\quad
B=\left(
\begin{matrix}
12&3+ I&5\\
3 - I&2&1 - I\\
5&1 + I&6
\end{matrix}
\right)
$$

In [6]:
A = Matrix([[4, 6, 10], [6, 13, 17], [10, 17, 62]])
B = Matrix([[12, 3 + I, 5], [3 - I, 2, 1 - I], [5, 1 + I, 6]])
LA = A.cholesky(hermitian=False)
LB = B.cholesky()
display(Latex(f'L_A = {latex(LA)},\ L_AL_A^T - A= {latex(simplify(LA * LA.T - A))},\\\\\
        B.is\_positive\_definite\ {B.is_positive_definite},\\\\\
        L_B = {latex(LB)},\\\\\
        simplify(L_B) = {latex(simplify(LB))},\\\\\
        simplify(L_B*L_B^H - B) = {latex(simplify(LB * LB.H - B))}'))

<IPython.core.display.Latex object>

### LDL разложение
$A = L D L^T$ для симметричной вещественной матрицы $A$

$A = L D L^H$ для положительно определенной эрмитовой матрицы $A$

$L$ - левая треугольная матрица,
$D$ - диагональная матрица.

В Sympy в классе матриц есть метод **LDLdecomposition**, возвращающий матрицы $D$ и $L$. В случае вещественной симметричной матрицы $A$ передаем методу необязательный параметр hermitian=False, по умолчанию равный True.
### Пример 3.
Построим  LDL разложение для матриц Примера 1

In [7]:
A = Matrix([[4, 6, 10], [6, 13, 17], [10, 17, 62]])
B = Matrix([[12, 3 + I, 5], [3 - I, 2, 1 - I], [5, 1 + I, 6]])
LA, DA = A.LDLdecomposition(hermitian=False)
LB, DB = B.LDLdecomposition()
LA, DA, LB, DB = [simplify(item) for item in (LA, DA, LB, DB)]
display(Latex("""A = {}\\\\A.is\_positive\_definite\ {}\\\\\
L_A = {}, D_A = {},\\\\L_AD_AL_A^T = {},\\\\\
B = {}\\\\\
B.is\_positive\_definite\ {}\\\\L_B = {}, D_B = {},\\\\\
L_BD_BL_B^H = {}\
""".format(*[latex(item) for item in (A, A.is_positive_definite,
                                      LA, DA, LA * DA * LA.T,
                                      B, B.is_positive_definite,
                                      LB, simplify(DB),
                                      simplify(LB * DB * LB.H))])))

<IPython.core.display.Latex object>

### LU разложение
$PA = LU$ для матрицы $A$

$L$ - левая треугольная матрица с единицами на главной диагонали,
$U$ - правая треугольная (трапециевидная) матрица,
$P$ - матрица перестановок.

$A = P^{-1}LU$.
### Пример 4.
Построим  LU разложение для матрицы
$$
M=\left(
\begin{matrix}
-2&3+ I&5 - 2I\\
 - I&2&1 - I\\
5&-1 + 4I&-3
\end{matrix}
\right)
$$

In [8]:
M = Matrix([[-2, 3 + I, 5 - 2 * I], [-I, 2, 1 - I], [5, -1 + 4 * I, -3]])
L, U, perm = M.LUdecomposition()
display(Latex("M = {}\\\\perm = {}, L = {}, U = {}\\\\LU = {}\
".format(*map(latex, (M, perm, *map(simplify, (L, U, L * U)))))))

<IPython.core.display.Latex object>

В Примере 3 не пришлось использовать перестановки, параметр perm, описывающий перестановки представляет собой пустой список.
### Пример 5.
Заменим в матрице $M$ элемент $-2$ на 0  и построим LU разложение для новой матрицы.

In [9]:
M[0, 0] = 0
L, U, perm = M.LUdecomposition()
display(Latex("M = {}\\\\perm = {}, L = {}, U = {}\\\\LU = {}\
".format(*map(latex, (M, perm, *map(simplify,  (L, U, L * U)))))))

<IPython.core.display.Latex object>

Произведение матриц LU отличается от исходной матрицы M перестановкой строк. Восстановим матрицу M, применяя перестановки в соответствии с результатом, выдаваемым LUdecomposition:

In [10]:
number_of_rows = M.shape[0]
MLU = simplify((L * U).permuteBkwd(perm))
P = eye(number_of_rows).permuteFwd(perm)
display(Latex(f"LU.permuteBkwd(perm) = {latex(MLU)}\\\\\
PM = LU\ {P * M == simplify(L * U)}"))

<IPython.core.display.Latex object>

LU разложение можно применять и для прямоугольной матрицы.
### Пример 6.
Добавим к матрице $M$ справа столбец из чисел 1, 2, 3  и построим LU разложение для новой матрицы.

In [13]:
number_of_rows = M.shape[0]
M = M.row_join(Matrix([k + 1 for k in range(number_of_rows)]))
L, U, perm = M.LUdecomposition()
MLU = simplify((L * U).permuteBkwd(perm))
L, U, LU, MLU = [simplify(item) for item in (L, U, L * U, MLU)]
P = eye(number_of_rows).permuteFwd(perm)
display(Latex("""M = {}\\\\perm = {}, L = {}, U = {}\\\\
LU = {}\\\\LU.permuteBkwd(perm) = {}\\\\PM = LU\ {}""".format(*map(latex, (M, perm, L, U, LU, MLU, P * M == LU)))))

<IPython.core.display.Latex object>

### QR разложение
$A = QR$ для симметричной вещественной матрицы $A$

$Q$ - матрица из ортогональных столбцов, т.е. $Q^HQ = I$, $I$ - единичная матрица, причем может не выполняться $QQ^H = I$ (для ортогональной матрицы $Q^HQ = QQ^H = I$),
$R$ - правая треугольная (трапециевидная) матрица.

Ранг матрицы $A$ равен числу столбцов матрицы $Q$.
### Пример 7.
Построим  QR разложение для матрицы Примера 5.

In [12]:
Q, R = M.QRdecomposition()
MQR = Q * R
Q, R, MQR = [simplify(item) for item in (Q, R, MQR)]
display(Latex("""M = {}\\\\Q = {}, R = {}\\\\
QR = {}\\\\M = QR\ {}""".format(*map(latex, (M, Q, R, MQR, M == MQR)))))

<IPython.core.display.Latex object>